# Exploratory analysis for SEO proofreading

The idea is to test the data for Cost and City pages, Test how can we extract the best context using RAG and then test different models and how good they perform (metrics would be required for better understanding).

In [3]:
import os

dir_path = os.getcwd()
dir_path


'/workspaces/seo-proofread-trustoo/experimentation'

In [4]:
import pandas as pd

def parse_excel_file(filename):
    try:
        # Read the Excel file
        data = pd.read_excel(filename)
        # Convert DataFrame to JSON
        json_data = data.to_json(orient='records')
        return data, json_data
    except Exception as e:
        print(f"Error parsing {filename}:", e)
        return None




In [5]:
# Parse the keyword files 
_,cost_keywords = parse_excel_file(f'{dir_path}/data/Kostenpagina - Schutting plaatsen - Keywords.xlsx')
print("Cost keywords data:", cost_keywords if cost_keywords is not None else "Failed to parse")

_,city_keywords = parse_excel_file(f'{dir_path}/data/Top 10 city page - Elektricien Haarlem - Keywords.xlsx')
print("\n\nCity keywords data:", city_keywords if city_keywords is not None else "Failed to parse")

# Parse the checklist templates
df_cost_checklist,cost_checklist = parse_excel_file(f'{dir_path}/data/SEO Content Checklist Cost Pages.xlsx')
print("\n\nCost checklist data:", cost_checklist if cost_checklist is not None else "Failed to parse")

df_city_checklist,city_checklist = parse_excel_file(f'{dir_path}/data/SEO Content Checklist Top 10 City Pages.xlsx')
print("\n\nCity checklist data:", city_checklist  if city_checklist is not None else "Failed to parse")

Cost keywords data: [{"Keyword":"goedkope schutting","Volume":1500.0,"KD":20},{"Keyword":"schutting plaatsen kosten","Volume":1100.0,"KD":2},{"Keyword":"schutting kopen en plaatsen","Volume":600.0,"KD":26},{"Keyword":"schutting laten plaatsen kosten","Volume":450.0,"KD":2},{"Keyword":"wat kost een schutting","Volume":400.0,"KD":1},{"Keyword":"kosten schutting","Volume":400.0,"KD":2},{"Keyword":"schutting inclusief plaatsen","Volume":300.0,"KD":4},{"Keyword":"schutting goedkoop","Volume":300.0,"KD":21},{"Keyword":"kosten schutting plaatsen","Volume":250.0,"KD":3},{"Keyword":"schutting kosten","Volume":200.0,"KD":5},{"Keyword":"kosten beton schutting per meter inclusief plaatsen","Volume":150.0,"KD":2},{"Keyword":"nieuwe schutting kosten","Volume":100.0,"KD":6},{"Keyword":"wat kost een schutting per meter","Volume":100.0,"KD":2},{"Keyword":"goedkope schutting plaatsen","Volume":100.0,"KD":12},{"Keyword":"wat kost een schutting plaatsen","Volume":90.0,"KD":2},{"Keyword":"goedkoopste schut

In [6]:
df_city_checklist.groupby('Category').head(1)

,Category,Checklist Item,Completed
0,Page Title & Meta Description,You can leave the page title & meta descriptio...,False
1,Headings & Keyword Research,"H2: Because it's a seperate section, it's good...",False
8,Internal Linking,Link to the Top 10 page of the main category.,False
12,General Quality Checks,Introduction should be to the point and only o...,False
25,Tone of Voice & Readability,"Use an active writing style, avoiding unnecess...",False
32,Formatting & Specific Guidelines,"Prices: Format as € 150,- / € 1.200,- / € 50,50.",False
38,Existing Pages Optimization,Check current ranking keywords and ensure they...,False


In [7]:
import docx
def read_docx(file_path: str) -> str:
    """Read the content of a .docx file."""
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

content = read_docx('./data/Kostenpagina - Schutting plaatsen - Content.docx')

In [8]:
import re
top10 = re.findall(r'\btop 10\b', content, re.IGNORECASE)
https_matches = re.findall(r'https://\S+', content)
print(https_matches,top10)

[] ['Top 10', 'TOP 10']


## Creating a wrapper around OpenRouter for multiple LLM usage

In [9]:
from typing import Any, List, Mapping, Optional
import requests
import json

from dotenv import load_dotenv

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

load_dotenv("../.env", override=True)

class MultiSelectCustomLLM(LLM):
    '''
    Can use multiple LLMs at the behest.
    # n: unique identifier
    llm_name: List[str] possible LLMs ('claude2','phi4','gemini2_5','gemini2_0','llama4') defaults to gemini 2.0
    '''
    n: int # echo from previous convo
    model:str = 'gemini2_0'
    # model_uri:str= self.getLLM()
    @property
    def _llm_type(self) -> str:
        return self.model
    
    def getLLM(self):
        LLMmap = {'claude2':'anthropic/claude-2',
                       'phi4':'microsoft/phi-4-reasoning-plus:free',
                       'gemini2_0':'google/gemini-2.0-flash-exp:free',
                       'gemini2_5':'google/gemini-2.5-pro-exp-03-25',
                       'llama4':'meta-llama/llama-4-scout:free'}
        return LLMmap.get(self.model,'google/gemini-2.0-flash-exp:free')

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:


        OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
        YOUR_SITE_URL = 'https://trustoo.nl' # trustoo.nl as http referer
        headers = {
            'Authorization': f'Bearer {OPENROUTER_API_KEY}',
            'HTTP-Referer': YOUR_SITE_URL,
            'Content-Type': 'application/json'
        }
        data = {
            'model': self.getLLM(), #self.model_uri,
            'messages': [
                {'role': 'user', 'content': prompt}
            ]
        }
        # Output example: {'choices': [{'message': {'role': 'assistant', 'content': "I am OpenAI's artificial intelligence model called GPT-3."}}], 'model': 'gpt-4-32k-0613', 'usage': {'prompt_tokens': 11, 'completion_tokens': 14, 'total_tokens': 25}, 'id': 'gen-e4MSuTT1v2wvrYFNFunhumsIawaI'}
        response = requests.post('https://openrouter.ai/api/v1/chat/completions', headers=headers, data=json.dumps(data))
        print(response.json())
        if 'error' in response.json():
            raise Exception(f"LLM Connection Error {response.json()['error']['code']}: {response.json()['error']['message']}")
        
        output = response.json()['choices'][0]['message']['content']

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n,"modelID":self.model,"modelURI":self.getLLM()}

In [10]:
llm = MultiSelectCustomLLM(n=1,model='phi4')
llm('what is your name LLM model?')

{'id': 'gen-1747063497-j4L0ToUnYzF9zHj6BJzg', 'provider': 'Chutes', 'model': 'microsoft/phi-4-reasoning-plus:free', 'object': 'chat.completion', 'created': 1747063498, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The user asks: "what is your name LLM model?" The question might be interpreted as asking "what is your name as an LLM model". Alternatively, I need to answer: exact chain-of-thought to answer what the name is that I identify as "LLM Model". I need to provide the following details: careful answer like "I am Phi ... I\'m a Large Language Model". The previous instructions about names: "I have a name" but indeed the assistant model is developed by Microsoft for handling "LLM"? The previous instructions: "what is your name LLM model?" Possibly the user means "what is the model name?" or "what is your calling name?" Answer: I might answer: "I am Phi, a large language model developed 

'The user asks: "what is your name LLM model?" The question might be interpreted as asking "what is your name as an LLM model". Alternatively, I need to answer: exact chain-of-thought to answer what the name is that I identify as "LLM Model". I need to provide the following details: careful answer like "I am Phi ... I\'m a Large Language Model". The previous instructions about names: "I have a name" but indeed the assistant model is developed by Microsoft for handling "LLM"? The previous instructions: "what is your name LLM model?" Possibly the user means "what is the model name?" or "what is your calling name?" Answer: I might answer: "I am Phi, a large language model developed by Microsoft, which builds on Open AI\'s work". But question is: "what is your name LLM model?" It can be answered: "I am a Phi-3.5, a model developed by Microsoft", but instructions: "what is your name LLM model?" The explicit mention "LLM model" might be referring to "Large Language Model", so my answer shoul

In [29]:
import time
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
#playing with prompt with same context and different models

prompt_testing = ChatPromptTemplate.from_template('''
    You are a SEO proofreading expert. You get Blog CITY or COST pages in docx with title, metadata and content. You will also get keyword metainformation like keyword, KD and Volume.
    Your task is to analyze the Blog and based on the checklist with Category, Checklist Item, you have to modify the Completed boolean.

    # Blog Type
    {blog_type}

    # Blog Content
    {content}

    # Keywords Analysis
    {keywords}

    # Checklist
    The checklist is as follows:
    {checklist}

    Produce the output as an excel file with all the checklist and Completed boolean either true or false based on Checklist item condition.

    Also propose 4-5 actionable recommendations for improving Blog Content.

    '''
)

prompt = prompt_testing.format_messages(
    blog_type='cost', #to do automatically  later
    content=content,
    keywords=str(cost_keywords),
    checklist=str(cost_checklist)
)
# prompt
blog_type='cost'
content=content
keywords=cost_keywords
checklist=cost_checklist

prompt_plain = f'''
    You are a SEO proofreading expert. You get Blog CITY or COST pages in docx with title, metadata and content. You will also get keyword metainformation like keyword, KD and Volume.
    Your task is to analyze the Blog and based on the checklist with Category, Checklist Item, you have to modify the Completed boolean.

    # Blog Type
    {blog_type}

    # Blog Content
    {content}

    # Keywords Analysis
    {keywords}

    # Checklist
    The checklist is as follows:
    {checklist}

    Produce the output as an excel file with all the checklist and Completed boolean either true or false based on Checklist item condition.

    Also propose 4-5 actionable recommendations for improving Blog Content.

    '''

# prompt_plain 'llama4',
res_list = dict()
for i in tqdm(('gemini2_5','gemini2_0','phi4')):
    llm = MultiSelectCustomLLM(n=1,model=str(i))
    res_list[i]=llm(prompt_plain)
    # res_list[i] = str(llm.invoke(prompt))
    print(res_list[i])
    time.sleep(6)

  0%|          | 0/3 [00:00<?, ?it/s]


KeyError: 'choices'

In [28]:
res_list

{'llama4': 'Based on the provided content, I\'ve evaluated the checklist items and completed the boolean values accordingly.\n\nHere is the output in a table format:\n\n| Category | Checklist Item | Completed |\n| --- | --- | --- |\n| Page Title | Start with the main keyword and include other relevant keywords. | False |\n| Page Title | Avoid repeating words within the title, as Google may remove duplicates. | False |\n| Page Title | Use ‘ | ’ to separate sections in the title. | True |\n| Page Title | End with ‘| Trustoo’ or ‘| Trustlocal’ if space allows, but prioritize relevant keywords. | False |\n| Meta Description | Include the main keyword. | False |\n| Meta Description | Clearly summarize the page content | True |\n| Meta Description | Encourage clicks with a CTA. | True |\n| Meta Description | For existing pages: check current meta description. | False |\n| Page Title & Meta Description | Check lenght with https://www.highervisibility.com/seo/tools/serp-snippet-optimizer/old/ 